In [7]:
-- Visualizando a base de dados
SELECT TOP(10) * FROM baseRH

(10 rows affected)

Total execution time: 00:00:00.019

Nome,Gênero,Data_Nascimento,Idade,Nível,Escolaridade,Área,Salário,Data_Admissão,Data_Demissão,Contrato,Motivo_Demissão,PCD,Alimentação,Saúde,Odont,Transporte,Creche,Combustível,Benefícios,Mês_admissão,Mês_demissão,Status,Faixa_Idade,Tempo_dias,Faixa_dias
Kylie Rame,Masculino,1996-03-17,28,Supervisor,Ensino Médio,Financeiro,7000,2019-01-01,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,24-34,364,270-364
Dwayne Betun,Masculino,1988-02-28,36,Gerente,Pós-graduação,Recursos Humanos,15000,2019-01-02,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,Férias,35-44,363,270-364
Chrisse Oxbrough,Masculino,1988-09-22,36,Gerente,Ensino Superior,Marketing,15000,2019-01-02,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,35-44,363,270-364
Kimmy Fardy,Feminino,1988-03-14,36,Gerente,Pós-graduação,Produção,15000,2019-01-02,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,35-44,363,270-364
Mata Steynor,Masculino,1994-06-28,30,Analista Sr,Ensino Técnico,Produção,5000,2019-01-02,NULL,CLT,NULL,NULL,600,600,600,600,NULL,500,2900,1,-,NULL,24-34,363,270-364
Rivi Ofen,Feminino,1981-10-19,43,Supervisor,Ensino Superior,Financeiro,7000,2019-01-03,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,35-44,362,270-364
Ross Tutchell,Masculino,1991-05-14,33,Assistente,Ensino Médio,Produção,2500,2019-01-03,NULL,CLT,NULL,NULL,600,600,600,600,500,NULL,2900,1,-,NULL,24-34,362,270-364
Lisa De Antoni,Feminino,1961-04-28,63,Estagiário,Ensino Médio,Administração,1200,2019-01-03,NULL,Estágio,NULL,NULL,200,200,200,200,NULL,NULL,800,1,-,NULL,55-64,362,270-364
Bethena Josephs,Feminino,1978-08-08,46,Analista Jr,Ensino Técnico,TI,3500,2019-01-03,NULL,CLT,NULL,NULL,600,600,600,600,NULL,NULL,2400,1,-,NULL,45-54,362,270-364
Kellia Koppel,Feminino,1970-07-19,54,Assistente,Ensino Superior,Recursos Humanos,2500,2019-01-03,NULL,CLT,NULL,NULL,600,600,600,600,NULL,NULL,2400,1,-,NULL,45-54,362,270-364


In [28]:
-- Total de funcionários por mês de admissão e MoM
WITH CTE AS
(
SELECT
    MONTH(Data_Admissão) AS 'Mês_admissão',
    COUNT(*) AS 'Total'
FROM baseRH
GROUP BY MONTH(Data_Admissão)
)

SELECT 
    Mês_admissão,
    Total,
    FORMAT(1.0 * Total/SUM(Total)OVER(), '0.00%') AS 'Percentual_total',
    FORMAT(1.0 * (Total - LAG(Total,1)OVER(ORDER BY Mês_admissão ASC))/Total, '0.00%') AS 'MoM'
FROM CTE
ORDER BY Mês_admissão ASC

(12 rows affected)

Total execution time: 00:00:00.163

Mês_admissão,Total,Percentual_total,MoM
1,103,"10,30%",NULL
2,68,"6,80%","-51,47%"
3,87,"8,70%","21,84%"
4,80,"8,00%","-8,75%"
5,90,"9,00%","11,11%"
6,91,"9,10%","1,10%"
7,89,"8,90%","-2,25%"
8,90,"9,00%","1,11%"
9,72,"7,20%","-25,00%"
10,82,"8,20%","12,20%"


O mês com a maior quantidade de contratações foi em Janeiro, no total 103 (10,30%) admitidos.

O mês com o maior crescimento de contratações foi em em Março com 21,84% em relação ao mês anterior.

In [1]:
-- 
SELECT 
    Gênero,
    COUNT(*) AS Total
FROM baseRH
GROUP BY Gênero

(2 rows affected)

Total execution time: 00:00:00.009

Gênero,Total
Masculino,509
Feminino,491


In [6]:
SELECT
    Área,
    AVG(Salário) AS 'Salario_medio',
    FORMAT(1.0 * AVG(Salário)/SUM(AVG(Salário))OVER(), '0.00%') AS 'Percentual_total'
FROM baseRH
GROUP BY Área
ORDER BY Salario_medio DESC

(9 rows affected)

Total execution time: 00:00:00.387

Área,Salario_medio,Percentual_total
Recursos Humanos,6360,"11,21%"
Vendas,6348,"11,19%"
Marketing,6339,"11,17%"
Financeiro,6326,"11,15%"
Administração,6314,"11,13%"
Operações,6308,"11,12%"
Produção,6281,"11,07%"
TI,6236,"10,99%"
Logística,6236,"10,99%"


In [32]:
SELECT 
    Idade,
    COUNT(*) AS 'Total'
FROM baseRH
GROUP BY Idade
ORDER BY Idade ASC

(41 rows affected)

Total execution time: 00:00:00.026

Idade,Total
24,31
25,22
26,24
27,18
28,19
29,29
30,28
31,25
32,19
33,18


In [33]:
SELECT 
    Faixa_Idade,
    COUNT(*) AS 'Total'
FROM baseRH
GROUP BY Faixa_Idade
ORDER BY Faixa_Idade ASC

(4 rows affected)

Total execution time: 00:00:00.021

Faixa_Idade,Total
24-34,258
35-44,250
45-54,252
55-64,240


In [36]:
SELECT 
    Idade,
    Salário
FROM baseRH

(1000 rows affected)

Total execution time: 00:00:00.015

Idade,Salário
28,7000
36,15000
36,15000
36,15000
30,5000
43,7000
33,2500
63,1200
46,3500
54,2500
